In [3]:
import pandas as pd
import re
from string import digits
import nltk
from fuzzywuzzy import fuzz
#nltk.download()

import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

pd.set_option('display.max_rows', 550)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_colwidth', 200)

--- 2.09808349609375e-05 seconds ---


In [5]:
#Combine Fragmented files into one single file


df6 = pd.read_csv('JUN17.csv')
df7 = pd.read_csv('JUL17.csv')
df8 = pd.read_csv('AUG17.csv')
df9 = pd.read_csv('SEP17.csv')
df10 = pd.read_csv('OCT17.csv')
df11 = pd.read_csv('NOV17.csv')
df12 = pd.read_csv('DEC17.csv')
df13 = pd.read_csv('JAN18.csv')
df14 = pd.read_csv('FEB18.csv')



#df = pd.read_csv('JUN17.csv')

#df = df6
print("--- %s seconds ---" % (time.time() - start_time))

FileNotFoundError: File b'JUN17.csv' does not exist

In [ ]:
df = df6.append([df7, df7, df8, df9, df10, df11, df12, df13, df14], ignore_index=True)
#df.to_csv('NOV17.csv')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df.head()

In [ ]:
"""

Preprocess:

Remove dupes
Remove http
Remove numbers
Remove all-too-freq tweets by user
Remove rows with less than X words
Remove adverts using fuzzy
Make lowercase
Remove stop words
Lemma
Stem

"""

In [ ]:
#remove unnecessary columns
df = df.drop(columns = ['to', 'geo', 'mentions', 'hashtags', 'id', 'permalink'])

In [ ]:
#Remove dupes
print(len(df))
df = df.drop_duplicates()
print(len(df))

In [ ]:
#Remove tweets with less than X words
count = df['text'].str.count(' ').add(1).value_counts(sort=False)

df = df.drop(df[df['text'].str.count(' ').add(1) < 10].index)
print(len(df))

In [ ]:
#Remove http
df['text'] = df['text'].str.replace('http://.*', '', regex=True)
df['text'] = df['text'].str.replace('https://.*', '', regex=True)

In [ ]:
#Remove numbers
df['text'] = df['text'].str.replace('\d+', '')

In [ ]:
#Remove @usernames
df['text'] = df['text'].str.replace('@\w{3,}', '', regex=True)

In [ ]:
#Remove symbols #, $ etc.

symbols_list = ['$', '#', '.', '\"', '-', ',', '/', '!', ':', ';', '?', '%', '&', '|', ')', '(', '+', '*', '”', '“']

for symbol in symbols_list:
    df['text'] = df['text'].str.replace(symbol, '')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#Make lowercase
df['text'] = df['text'].str.lower()

In [ ]:
#Lemma?

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
#str.split tokenizes, kind of?
#df['text'] = df['text'].str.split()

In [ ]:
username_set = set(df['username'])
unique_username_list = list(username_set)
print(len(unique_username_list))

In [ ]:
len(df)

In [ ]:
df.sample(5)

In [ ]:
#Export user list to csv for processing in user_query file
unique_username_list = pd.DataFrame(unique_username_list)
#Export user list to csv for processing in user_query file
unique_username_list = pd.DataFrame(unique_username_list)
unique_username_list = unique_username_list.rename(columns = {0:'username'})
unique_username_list = list(unique_username_list['username'])
unique_username_list

In [ ]:
#Write to file
#with open('unique_username_list_JUN.csv', mode='wt', encoding='utf-8') as myfile:
#    myfile.write('\n'.join(unique_username_list))
#    myfile.write('\n')

In [ ]:
unique_username_list

In [ ]:
#Find number of tweets per user
username_set = set(df['username'])
unique_username_list = list(username_set)
print(len(unique_username_list))
print(len(df))

In [ ]:
df2 = df[['text', 'username']].groupby('username').count().reset_index().sort_values(['text'], ascending=False)

In [ ]:
#delete tweets whose usernames have more than 2k tweets
df2 = df2.drop(df2[df2['text'] < 2000].index)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
username_list = list(df2['username'])
#identify those with greater than 2000. Once identified, delete them from the original df.
for user in username_list:
    df = df.drop(df[df['username'] == user].index)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
username_set = set(df['username'])
unique_username_list = list(username_set)
print(len(unique_username_list))
print(len(df))

In [ ]:
#Get list of users with more than 800 tweets
df2 = df[['text', 'username']].groupby('username').count().reset_index().sort_values(['text'], ascending=False)
df2 = df2.drop(df2[df2['text'] < 500].index)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#username_list = list(df2['username'])
#for user in username_list:
#    df = df.drop(df[df['username'] == user].index)
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
username_set = set(df['username'])
unique_username_list = list(username_set)
print(len(unique_username_list))
print(len(df2))

In [ ]:
df2.head(200)

In [ ]:
#Remove adverts using Fuzzy
fuzz_ratio = 0

username_set = set(df2['username'])
unique_username_list = list(username_set)
print(len(unique_username_list))

for user in unique_username_list:
    single_user_df = df.loc[df['username'] == user]
    #print(single_user_df)
    #pick two random tweets from a given user, apply fuzzy ratio, iterate 4 times and then average to find overall similarity between tweets.
    for i in range(24):
        two_tweets = single_user_df.sample(n=2)
        #print(two_tweets['text'])
        fuzz_ratio += fuzz.ratio(two_tweets['text'].iloc[0], two_tweets['text'].iloc[1])
        fuzz_ratio = fuzz_ratio / 24
    print(fuzz_ratio, user)
    if fuzz_ratio > 1.4:
        df = df.drop(df[df['username'] == user].index)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
username_set = set(df['username'])
unique_username_list = list(username_set)
print(len(unique_username_list))
print(len(df))

In [ ]:
df['month'] = pd.DatetimeIndex(df['date']).month

In [ ]:
df['text'].isnull().sum()


df = df.drop(df[df['text'].isnull()].index)
df['text'].isnull().sum()
df.head()

In [ ]:
#Other pre-processing
df = df.drop(df[df['text'] == 'what it feels like to earn bitcoin on a daily basis with '].index)
print(len(df))
df = df.drop(df[df['text'].str.contains("news")].index)
print(len(df))
df = df.drop(df[df['text'].str.contains("wcx")].index)
print(len(df))

In [ ]:
df.to_csv('ALL1718_PXD.csv')

In [ ]:
df = pd.read_csv('ALL1718_PXD.csv', parse_dates=['date'], 
    date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
print(type(df['date']))
df.to_csv('ALL1718_PXD.csv')

In [ ]:
#Start Viz
import matplotlib.pyplot as plt 
plot_df = df[['Topic', 'month','date']].groupby(['month', 'Topic']).count().reset_index()

In [ ]:
plot_df

In [ ]:
for month in range(6,13):
    print(plot_df[['date', 'Topic']][plot_df['month'] == month])
    #print(topic)

In [ ]:
fig , ax = plt.subplots()

for i in plot_df.groupby('Topic'):
    print(i[1])
    a = pd.DataFrame(i[1])[['month','date']]
    a.plot(ax = ax)

In [ ]:
a